# Splunk App for Data Science and Deep Learning - Notebook Template 

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with you Splunk platform by utilizing the Splunk App for Data Science and Deep Learning (DSDL) - formerly known as the Deep Learning Toolkit for Splunk (DLTK). Find more examples and information in the app and on the [DSDL splunkbase page](https://splunkbase.splunk.com/app/4607/#/details).

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os

import time

import pymilvus

from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("pymilvus version: " + pymilvus.__version__)

numpy version: 1.23.5
pandas version: 2.0.2
pymilvus version: 2.2.9


## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DSDL Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DSDL app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter.

### Option 1 - pull data from Splunk

In [5]:
from dsdlsupport import SplunkSearch as SplunkSearch

In [6]:
search = SplunkSearch.SplunkSearch()

In [7]:
df = search.as_df()
df

,_time,feature_0,feature_1,feature_2,i,s
0,2022-10-12T15:58:22.000+02:00,0.2428163280710578,0.2446760144084692,0.878678847104311,1,1
1,2022-10-12T15:58:22.000+02:00,0.458672939799726,0.4403486642986536,0.5951908086426556,2,2
2,2022-10-12T15:58:22.000+02:00,0.4268499561585486,0.70259142620489,0.6676120953634381,3,0
3,2022-10-12T15:58:22.000+02:00,0.9911085437051952,0.7537638866342604,0.8423880040645599,4,1
4,2022-10-12T15:58:22.000+02:00,0.2864162167534232,0.4240174447186291,0.7597004366107285,5,2
5,2022-10-12T15:58:22.000+02:00,0.9403447071090341,0.10794003773480654,0.22778158355504274,6,0
6,2022-10-12T15:58:22.000+02:00,0.6541405455209315,0.7657363642938435,0.07344387145712972,7,1
7,2022-10-12T15:58:22.000+02:00,0.8196326880715787,0.34731235541403294,0.8556739939376712,8,2
8,2022-10-12T15:58:22.000+02:00,0.3521619248203933,0.04771727975457907,0.07742329593747854,9,0
9,2022-10-12T15:58:22.000+02:00,0.2476972616277635,0.5393188064917922,0.7553935945034027,10,1


### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

| makeresults count=10<br>
| streamstats c as i <br>
| eval s = i%3 <br>
| eval feature_{s}=0 <br>
| foreach feature_* [eval \<\<FIELD\>\>=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone_template _time feature_* i into app:barebone_template<br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [17]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("push_to_milvus_ua")

In [18]:
df

,user_agent,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,...,vector_22,vector_23,vector_24,vector_25,vector_26,vector_27,vector_28,vector_29,vector_30,vector_31
0,Mozilla/5.0 (Linux; U; Android 4.0.4; en-us; G...,1,0,2,0,0,0,0,0,0,...,0,3,1,0,2,2,0,0,1,2
1,Mozilla/5.0 (iPad; CPU OS 7_0 like Mac OS X) A...,0,0,0,0,0,0,0,0,1,...,0,1,1,1,1,1,2,0,0,0
2,Mozilla/5.0 (Windows NT 10.0; Win64; x64; Trid...,0,0,0,2,1,0,0,0,2,...,0,0,1,0,4,0,0,0,0,0
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...,0,0,2,0,0,0,0,0,0,...,0,0,1,3,2,0,0,0,2,0
4,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,0,0,0,2,1,0,0,0,1,...,0,0,1,0,4,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Mozilla/5.0 (Linux; Android 4.1.2; Panasonic T...,0,0,3,1,0,0,0,0,0,...,1,1,2,2,3,1,0,1,0,1
9996,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,0,0,2,1,1,0,0,1,1,...,0,1,1,0,7,0,2,1,1,0
9997,Mozilla/5.0 (X11; U; Linux x86_64; en-gb) Appl...,0,0,1,0,0,1,1,0,0,...,1,1,2,2,3,0,0,1,0,1
9998,Mozilla/5.0 (Linux; U; Android 4.1.2; en-US; P...,1,0,1,1,0,0,0,0,0,...,1,1,1,0,2,4,1,0,2,2


In [19]:
param

{'options': {'params': {'mode': 'stage',
   'algo': 'push_to_milvus',
   'collection_name': 'user_agent_collection',
   'n_dims': '32',
   'label_field_name': 'user_agent'},
  'args': ['user_agent', 'vector_*'],
  'feature_variables': ['user_agent', 'vector_*'],
  'model_name': 'push_to_milvus_ua',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '600',
   'max_inputs': '100000',
   'max_memory_usage_mb': '4000',
   'max_model_size_mb': '30',
   'max_score_time': '600',
   'use_sampling': 'true'},
  'kfold_cv': None},
 'feature_variables': ['user_agent',
  'vector_0',
  'vector_1',
  'vector_2',
  'vector_3',
  'vector_4',
  'vector_5',
  'vector_6',
  'vector_7',
  'vector_8',
  'vector_9',
  'vector_10',
  'vector_11',
  'vector_12',
  'vector_13',
  'vector_14',
  'vector_15',
  'vector_16',
  've

## Stage 2 - create and initialize a model

In [20]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}

    try:
        n_dims=int(param['options']['params']['n_dims'])
    except:
        n_dims=8
        
    pk_type=DataType.VARCHAR        
    embedding_type=DataType.FLOAT_VECTOR
    
    try:
        collection_name=param['options']['params']['collection_name']
    except:
        collection_name="default_collection"
    
    print("start connecting to Milvus")
    connections.connect("default", host="172.17.0.1", port="19530")

    collection_exists = utility.has_collection(collection_name)
        
    fields = [
        FieldSchema(name="_key", is_primary=True, auto_id=True, dtype=DataType.INT64),
        FieldSchema(name="embeddings", dtype=embedding_type, dim=n_dims),
        FieldSchema(name="label", dtype=DataType.VARCHAR, max_length=1000),
    ]

    schema = CollectionSchema(fields, f"dsdl schema for {collection_name}")
    
    if collection_exists:
        print(f"The collection {collection_name} already exists")
        collection = Collection(collection_name)
        collection.load()
    else:
        print(f"The collection {collection_name} does not exist")
        print(f"creating new collection: {collection_name}")
        collection = Collection(collection_name, schema, consistency_level="Strong")
        index = {
            "index_type": "IVF_FLAT",
            "metric_type": "L2",
            "params": {"nlist": 1024},
        }
        collection.create_index("embeddings", index)
    
    model['collection']=collection
    model['collection_name']=collection_name

    return model

In [21]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

start connecting to Milvus
The collection user_agent_collection already exists
{'collection': <Collection>:
-------------
<name>: user_agent_collection
<description>: dsdl schema for user_agent_collection
<schema>: {'auto_id': True, 'description': 'dsdl schema for user_agent_collection', 'fields': [{'name': '_key', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 32}}, {'name': 'label', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000}}]}
, 'collection_name': 'user_agent_collection'}


## Stage 3 - fit the model

In [28]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    df=df.copy()
    label_field_name=param['options']['params']['label_field_name']
    label_column = df[label_field_name].astype(str)
    df.drop(label_field_name, axis=1, inplace=True)
    df_list=df.values.tolist()
    data=[ df_list, label_column.tolist() ]
    model['collection'].insert(data)    
    info = {"message": f"inserted data to collection {model['collection_name']}"}
    return df

In [29]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

      vector_0  vector_1  vector_2  vector_3  vector_4  vector_5  vector_6  \
0            1         0         2         0         0         0         0   
1            0         0         0         0         0         0         0   
2            0         0         0         2         1         0         0   
3            0         0         2         0         0         0         0   
4            0         0         0         2         1         0         0   
...        ...       ...       ...       ...       ...       ...       ...   
9995         0         0         3         1         0         0         0   
9996         0         0         2         1         1         0         0   
9997         0         0         1         0         0         1         1   
9998         1         0         1         1         0         0         0   
9999         0         0         2         0         0         0         0   

      vector_7  vector_8  vector_9  ...  vector_22  vector_23  

## Stage 4 - apply the model

In [30]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    return df

In [31]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

                                             user_agent  vector_0  vector_1  \
0     Mozilla/5.0 (Linux; U; Android 4.0.4; en-us; G...         1         0   
1     Mozilla/5.0 (iPad; CPU OS 7_0 like Mac OS X) A...         0         0   
2     Mozilla/5.0 (Windows NT 10.0; Win64; x64; Trid...         0         0   
3     Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...         0         0   
4     Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...         0         0   
...                                                 ...       ...       ...   
9995  Mozilla/5.0 (Linux; Android 4.1.2; Panasonic T...         0         0   
9996  Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...         0         0   
9997  Mozilla/5.0 (X11; U; Linux x86_64; en-gb) Appl...         0         0   
9998  Mozilla/5.0 (Linux; U; Android 4.1.2; en-US; P...         1         0   
9999  Mozilla/5.0 (Linux; U; Android 4.0.4; en-us; E...         0         0   

      vector_2  vector_3  vector_4  vector_5  vecto

## Stage 5 - save the model

In [1]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    return model

## Stage 6 - load the model

In [2]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    return model

## Stage 7 - provide a summary of the model

In [17]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code